<a href="https://colab.research.google.com/github/alanland/colab-notebooks/blob/main/ChuanhuChatGPT/ChuanhuChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ChuanhuChatGPT: Lightweight and User-friendly Web-UI for LLMs**


Streaming / Unlimited conversations / Save history / Preset prompts / Chat with files / Web search
LaTeX rendering / Table rendering / Code highlighting
Auto dark mode / Adaptive web interface / WeChat-like theme
Multi-parameters tuning / Multi-API-Key support / Multi-user support
Compatible with GPT-4 / Local deployment for LLMs

https://github.com/GaiZhenbiao/ChuanhuChatGPT


In [2]:
### make sure that CUDA is available in Edit -> Nootbook settings -> GPU
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15360 MiB, 15101 MiB


## Installnation

In [3]:
!cd /content
# !git clone https://github.com/GaiZhenbiao/ChuanhuChatGPT.git
!git clone https://github.com/alanland/ChuanhuChatGPT.git
%cd ChuanhuChatGPT

Cloning into 'ChuanhuChatGPT'...
remote: Enumerating objects: 2849, done.
remote: Counting objects: 100% (1450/1450), done.
remote: Compressing objects: 100% (294/294), done.
remote: Total 2849 (delta 1255), reused 1182 (delta 1156), pack-reused 1399
Receiving objects: 100% (2849/2849), 1.38 MiB | 11.66 MiB/s, done.
Resolving deltas: 100% (1931/1931), done.
/content/ChuanhuChatGPT


In [ ]:
!git checkout main-local
!git pull
!pip install -r requirements.txt

In [ ]:
!pip install -r requirements_advanced.txt

In [ ]:
# ChatGLM-6B
!pip install protobuf==3.20.0
!pip install torch>=1.10
!pip install gradio
!pip install mdtex2html
!pip install sentencepiece
# end ChatGLM-6B

In [ ]:
!git pull
!python ChuanhuChatbot.py

Already up to date.
2023-06-01 13:58:39,211 [INFO] [models.py:562] 正在加载OpenAI模型: gpt-3.5-turbo
2023-06-01 13:58:39,211 [INFO] [models.py:615] Model is set to:  gpt-3.5-turbo
2023-06-01 13:58:39,394 [INFO] [utils.py:410] Your IP region: United States。
2023-06-01 13:58:39,511 [INFO] [ChuanhuChatbot.py:460] 
川虎的温馨提示：访问 http://localhost:7860 查看界面
Reloading javascript...
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://8806487def2479f47c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
2023-06-01 13:58:53,558 [INFO] [models.py:562] 正在加载OpenAI模型: gpt-3.5-turbo
2023-06-01 13:58:53,559 [INFO] [models.py:615] Model is set to:  gpt-3.5-turbo
2023-06-01 13:58:55,718 [INFO] [models.py:562] 正在加载OpenAI模型: gpt-3.5-turbo
2023-06-01 13:58:55,718 [INFO] [models.py:615] Model is set to:  gpt-3.5-turbo
2023-06-01 13:59:25,931 [INFO] [models.py:572] 正在加载ChatGLM模型: chatglm-6b-int4
